In [ ]:
import numpy as np

from dataset import Dataset

## 2.1: Implementing the recurrent neural network
The vanilla neural network we will implement has three hyperparameters - the size of the hidden layer, the number of timesteps and the learning rate. The optimization algorithm which you will implement is Adagrad.

With the assumption of single precision of decimal numbers (4 bytes), and the size of the hidden layer of 500, the language modelling task with a vocabulary size of 70 (input and output dimension) - how large is the memory requirement of the parameters of the model? Which parameter takes up the most memory? Repeat the same exercise with a vocabulary of size 10000.


In [ ]:
float_t = 4 # 4 bytes
hidden_layer_size = 500
vocabulary_size = 70 #10000
input_dimension = vocabulary_size
output_dimension = vocabulary_size

'''
Parameters are: U, V, W
'''
print('Paramteres are: U, V, W, biases: b, c')
print()

w_size = hidden_layer_size*hidden_layer_size
print('W.size = {}*{} = {}'.format(hidden_layer_size, hidden_layer_size, w_size))
u_size = input_dimension*hidden_layer_size
print('U.size = {}*{} = {}'.format(input_dimension, hidden_layer_size, u_size))
v_size = hidden_layer_size*output_dimension
print('V.size = {}*{} = {}'.format(hidden_layer_size, output_dimension, v_size))
b_size = vocabulary_size
print('b.size = {}'.format(vocabulary_size))
c_size = output_dimension
print('c.size = {}'.format(c_size))
total_size = w_size+u_size+v_size+b_size+c_size
print('total size = {}+{}+{}+{}+{} = {}'.format(w_size,u_size,v_size,b_size,c_size, total_size))
print('total size in bytes = {}*{} = {}'.format(total_size, float_t, total_size*float_t))

The aforementioned vocabulary sizes are the one you will use in the task of character level language modelling (approx. 70), and for word level language models (commonly much larger than $10^5$).

The initial hyperparameters are the hidden layer size of 100, an unroll of 30 timesteps and a learning rate of 1e-1. You should try out some other hyperparameters as well and report on loss and sample convergence.

The implementation of the recurrent neural network can be split conceptually into four parts, as follows:

### 2.1.1: Parameter initialization

The parameter matrices should be initialized randomly from a gaussian distribution with a standard deviation of 1e-2. The bias vectors should be initialized to zeros.

Parameter initializtion code could look as follows:

```
def __init__(self, hidden_size, sequence_length, vocab_size, learning_rate):
    ...
```

Remember to add a redundant dimension while initializing bias vectors for numpy broadcasting.

### 2.1.2: Forward pass

The forward pass of the recurrent neural network can be imagined as a loop in which we iteratively perform a single timestep. The forward pass can therefore be implemented as a function which processes a single timestep, and a function which iterates over the whole sequence and stores the results.

The skeletons of the aforementioned functions could look as follows:

```
def rnn_step_forward(self, x, h_prev, U, W, b):
    ...

def rnn_forward(self, x, h0, U, W, b):
    ...
```

Notice that even though the parameters U, W and b exist as instance variables of the class (self.U, self.W, self.b), due to reproducibility we redundantly leave them as arguments of the functions.

### 2.1.3: Backward pass

The backward pass of the recurrent network is conceptually similar to the forward pass with the iteration being done in reverse, from the last timestep towards the first one. The backward pass is done by the backpropagation through time algorithm (BPTT). The skeletons of the BPTT algorithm functions could look as follows:

```
def rnn_step_backward(self, grad_next, cache):
    ...
    
def rnn_backward(self, dh, cache):
    ...
```

In order to mitigate the exploding gradient problem, before applying the accumulated gradient, apply elementwise gradient clipping. We recommend you use the `np.clip()` method, and leave the values in the interval of $[−5,5]$.

### 2.1.4: Training loop

The training loop connects the recurrent network with the input data, calculates the loss and does parameter optimization. Ideally, the network should be idependent of the data - so it is recommended to separate the iteration over batches and epochs from the network class.

A skeleton implementation of the loss computation function could look as follows:

```
def output(h, V, c):
    ...

def output_loss_and_grads(self, h, V, c, y):
    ...
```

The implementation of the parameter learning function could look as follows:

```
def update(self, dU, dW, db, dV, dc,
    ...
```

The implementation of the control flow loop and iterating the optimization process could look as follows:

```
def run_language_model(dataset, max_epochs, hidden_size=100, sequence_length=30, learning_rate=1e-1, sample_every=100)
    ...
```

The `step` function, whose skeleton is not described, contains the logic of running a single forward and backward pass of a neural network on a minibatch of inputs. As inputs, the method accepts an initial hidden state vector, the one-hot encoded inputs and outputs of shapes BxTxV, where B is the minibatch size, T the number of timesteps to unroll the network for and V the size of the vocabulary. As outputs we receive the loss in that step as well as the hidden state from the last timestep, which should then be used as the next initial hidden state.

Important: Think about how the data has to be aligned in order for the last hidden states from the previous step to be valid initial hidden states for the current step.



In [ ]:
# @lab1:
def softmax(_input):
    expinput = np.exp(_input)
    sumexp = expinput.sum(axis=1)
    return (expinput.transpose() / sumexp).transpose()

## @lab0
# stable softmax
def stable_softmax(x):
    exp_x_shifted = np.exp(x - np.max(x))
    probs = exp_x_shifted / np.sum(exp_x_shifted)
    return probs

softmax = stable_softmax

In [ ]:

class RNN:
    def __init__(self, hidden_size, sequence_length, vocab_size, learning_rate):
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.learning_rate = learning_rate

        # numpy.random.normal(loc=0.0, scale=1.0, size=None)
        # Draw random samples from a normal (Gaussian) distribution.
        _input_size = self.vocab_size # variables just to better understand
        _output_size = self.vocab_size
        self.U = np.random.normal(size=[_input_size, hidden_size], scale=1.0 / np.sqrt(hidden_size)) # ... input projection
        self.W = np.random.normal(size=[hidden_size, hidden_size], scale=1.0 / np.sqrt(hidden_size)) # ... hidden-to-hidden projection
        self.b = np.zeros((1, self.hidden_size)) # ... input bias

        self.V = np.random.normal(size=[hidden_size, _output_size], scale=1.0 / np.sqrt(_output_size)) # ... output projection
        self.c = np.zeros((1, self.vocab_size)) # ... output bias

        # memory of past gradients - rolling sum of squares for Adagrad
        self.memory_U, self.memory_W, self.memory_V = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
        self.memory_b, self.memory_c = np.zeros_like(self.b), np.zeros_like(self.c)


    def rnn_step_forward(self, x, h_prev, U, W, b):
        # A single time step forward of a recurrent neural network with a 
        # hyperbolic tangent nonlinearity.

        # x - input data (minibatch size x input dimension)
        # h_prev - previous hidden state (minibatch size x hidden size)
        # U - input projection matrix (input dimension x hidden size)
        # W - hidden to hidden projection matrix (hidden size x hidden size)
        # b - bias of shape (hidden size x 1)

        h_current = np.tanh(np.dot(h_prev, W) + np.dot(x, U) + b)
        cache = (x, h_prev, h_current)

        # return the new hidden state and a tuple of values needed for the backward step

        return h_current, cache


    def rnn_forward(self, x, h0, U, W, b):
        # Full unroll forward of the recurrent neural network with a 
        # hyperbolic tangent nonlinearity

        # x - input data for the whole time-series (minibatch size x sequence_length x input dimension)
        # h0 - initial hidden state (minibatch size x hidden size)
        # U - input projection matrix (input dimension x hidden size)
        # W - hidden to hidden projection matrix (hidden size x hidden size)
        # b - bias of shape (hidden size x 1)

        h = [h0]
        cache = []
        for seq in range(self.sequence_length):
            input_data = x[:, seq, :]
            h_current, cache_current = self.rnn_step_forward(input_data, h[-1], U, W, b)
            h.append(h_current)
            cache.append(cache_current)

        # return the hidden states for the whole time series (T+1) and a tuple of values needed for the backward step
        #h = h[1:] # omit initial state h0
        h = np.array(h[1:]).transpose((1, 0, 2))
        return h, cache


    def rnn_step_backward(self, grad_next, cache):
        # A single time step backward of a recurrent neural network with a 
        # hyperbolic tangent nonlinearity.

        x, h_prev, h_current = cache
        # grad_next - upstream gradient of the loss with respect to the next hidden state and current output
        ## derivative of tanh(x) = 1-tanh^2(x)
        ## http://math2.org/math/derivatives/more/hyperbolics.htm
        grad_next = grad_next * (1 - h_current*h_current)

        # cache - cached information from the forward pass

        ##dh_prev, dU, dW, db = None, None, None, None
        dh_prev = np.dot(grad_next, self.W.transpose())
        dU = np.dot(x.transpose(), grad_next)
        dW = np.dot(h_prev.transpose(), grad_next)
        db = np.sum(grad_next, axis=0)

        # compute and return gradients with respect to each parameter
        # HINT: you can use the chain rule to compute the derivative of the
        # hyperbolic tangent function and use it to compute the gradient
        # with respect to the remaining parameters
        
        '''
        In order to mitigate the exploding gradient problem, before applying the accumulated
        gradient, apply elementwise gradient clipping. We recommend you use the np.clip()
        method, and leave the values in the interval of [−5,5].
        '''
        dh_prev = np.clip(dh_prev, -5, 5)
        dU = np.clip(dU, -5, 5)
        dW = np.clip(dW, -5, 5)
        db = np.clip(db, -5, 5)

        return dh_prev, dU, dW, db


    def rnn_backward(self, dh, cache):
        # Full unroll forward of the recurrent neural network with a 
        # hyperbolic tangent nonlinearity

        ###dU, dW, db = None, None, None

        dh_prev = 0
        dU_sum = 0
        dW_sum = 0
        db_sum = 0

        for dh_elem, cache_elem in reversed(list(zip(dh, cache))):
            dh_prev, dU, dW, db = self.rnn_step_backward(dh_elem + dh_prev, cache_elem)
            dU_sum += dU
            dW_sum += dW
            db_sum += db
        return dU_sum, dW_sum, db_sum

        # compute and return gradients with respect to each parameter
        # for the whole time series.
        # Why are we not computing the gradient with respect to inputs (x)?
        #return dU, dW, db


    def output(self, h, V, c):
        # Calculate the output probabilities of the network
        return softmax(np.dot(h, V) + c)


    def output_loss_and_grads(self, h, V, c, y):
        # Calculate the loss of the network for each of the outputs

        # h - hidden states of the network for each timestep. 
        #     the dimensionality of h is (batch size x sequence length x hidden size (the initial state is irrelevant for the output)
        # V - the output projection matrix of dimension hidden size x vocabulary size
        # c - the output bias of dimension vocabulary size x 1
        # y - the true class distribution - a tensor of dimension 
        #     batch_size x sequence_length x vocabulary size - you need to do this conversion prior to
        #     passing the argument. A fast way to create a one-hot vector from
        #     an id could be something like the following code:

        #   y[batch_id][timestep] = np.zeros((vocabulary_size, 1))
        #   y[batch_id][timestep][batch_y[timestep]] = 1

        #     where y might be a list or a dictionary.

        #loss, dh, dV, dc = None, None, None, None
        loss = 0.0
        dh = []
        dV = np.zeros_like(self.V)
        dc = np.zeros_like(self.c)

        # calculate the output (o) - unnormalized log probabilities of classes
        # calculate yhat - softmax of the output
        # calculate the cross-entropy loss
        # calculate the derivative of the cross-entropy softmax loss with respect to the output (o)
        # calculate the gradients with respect to the output parameters V and c
        # calculate the gradients with respect to the hidden layer h
        batch_size = len(h)

        for t in range(self.sequence_length):
            yp = y[:, t, :]
            h_t = h[:, t, :]

            o = self.output(h_t, V, c)
            s = softmax(o)

            loss += -np.sum(np.log(s)*yp) / batch_size
            dO = (s - yp) / batch_size

            dV += np.dot(h_t.T, dO)
            dc += np.sum(dO, axis=0)

            dh_t = np.dot(dO, V.T)
            dh.append(dh_t)

        return loss, dh, dV, dc


    # The inputs to the function are just indicative since the variables are mostly present as class properties
    def update(self, dU, dW, db, dV, dc,
                     U, W, b, V, c,
                     memory_U, memory_W, memory_b, memory_V, memory_c):

        # update memory matrices
        memory_U += np.square(dU)
        memory_W += np.square(dW)
        memory_b += np.square(db)
        memory_V += np.square(dV)
        memory_c += np.square(dc)

        # perform the Adagrad update of parameters
        epsilon = 1e-6
        U -= self.learning_rate * dU / np.sqrt(memory_U + epsilon)
        W -= self.learning_rate * dW / np.sqrt(memory_W + epsilon)
        b -= self.learning_rate * db / np.sqrt(memory_b + epsilon)
        V -= self.learning_rate * dV / np.sqrt(memory_V + epsilon)
        c -= self.learning_rate * dc / np.sqrt(memory_c + epsilon)


    def step(self, h0, x_oh, y_oh):
        '''
        The step function, whose skeleton is not described, contains the logic
        of running a single forward and backward pass of a neural network on a minibatch
        of inputs. As inputs, the method accepts an initial hidden state vector, the
        one-hot encoded inputs and outputs of shapes BxTxV, where B is the minibatch
        size, T the number of timesteps to unroll the network for and V the size of the
        vocabulary. As outputs we receive the loss in that step as well as the hidden
        state from the last timestep, which should then be used as the next initial hidden state.
        '''
        h_next, cache = self.rnn_forward(x_oh, h0, self.U, self.W, self.b)
        loss, dh, dV, dc = self.output_loss_and_grads(h_next, self.V, self.c, y_oh)
        dU, dW, db = self.rnn_backward(dh, cache)
        self.update(dU, dW, db, dV, dc,
                    self.U, self.W, self.b, self.V, self.c,
                    self.memory_U, self.memory_W, self.memory_b, self.memory_V, self.memory_c)
        return loss, h_next[:, -1, :]


    # ...
    # code not necessarily nested in class definition
    def sample(self, seed, n_sample):
        ##h0, seed_onehot, sample = None, None, None 
        # inicijalizirati h0 na vektor nula
        # seed string pretvoriti u one-hot reprezentaciju ulaza
        h0 = np.zeros((dataset.batch_size, self.hidden_size))

        # seed to one hot
        seed_onehot = to_one_hot(seed, self.vocab_size)

        sample = seed

        return sample

    

## 2.2: Sampling data from the learned network

When training the network, we implicitly take the next character as the next input instead of the one predicted by the network. This causes our loss to be artificially smaller, and is not applicable during testing (when we do not know the correct character).

In the scope of the lab assignment we will not dive deep into sequence sampling, but implement the bare minimum. Every `sample_every` minibatches, you should run the sampling method from our recurrent neural network in the following fashion:

1. Store the current hidden state of the network (`h_train`)
2. Initialize an empty hidden state for sampling (`h0_sample`)
3. Define a seed sequence of symbols to "warm up" the network - ex: `seed ='HAN:\nIs that good or bad?\n\n'`
4. Define how many characters you will sample (`n_sample=300`)
5. Run the forward pass on the seed sequence "seed"
6. Sample the remaining `n_sample` - `len(seed)` characters by using roulette-wheel (probability-proportional) sampling

The skeleton of the sampling function could look as follows:

```
def sample(seed, n_sample):
    ...
```

In [ ]:

# @lab1:
'''
    # instantiate the data X and the labels Yoh_
    X = np.array([[1,3],[1,1],[3,2],[3,3]])
    Y_ = np.array([1,0,2,2])

    # Yoh c1 c2 c3
    #  x1 0  1  0
    #  x2 1  0  0
    #  x3 0  0  1
    #  x4 0  0  1

    Yoh_ = np.zeros((len(X), len(np.bincount(Y_))))
    Yoh_[range(len(Y_)), Y_] = 1
'''
def to_one_hot(x, vocab_size):
    x_onehot = np.zeros((len(x), vocab_size))
    x_onehot[range(len(x)), x] = 1
    return x_onehot

def run_language_model(dataset, max_epochs, hidden_size=100, sequence_length=30, learning_rate=1e-1, sample_every=100):

    vocab_size = len(dataset.sorted_chars)
    rnn = RNN(hidden_size, sequence_length, vocab_size, learning_rate) # initialize the recurrent network

    current_epoch = 0 
    batch = 0

    h0 = np.zeros((dataset.batch_size, hidden_size))

    average_loss = 0

    while current_epoch < max_epochs: 
        e, x, y = dataset.next_minibatch()

        if e: 
            current_epoch += 1
            h0 = np.zeros((dataset.batch_size, hidden_size))
            # why do we reset the hidden state here?
            ## new sequence start

        # One-hot transform the x and y batches
        ## list comprehension because of batch size - several arrays in one unit
        x_oh = np.array([to_one_hot(_x, vocab_size) for _x in x])
        y_oh = np.array([to_one_hot(_y, vocab_size) for _y in y])

        # Run the recurrent network on the current batch
        # Since we are using windows of a short length of characters,
        # the step function should return the hidden state at the end
        # of the unroll. You should then use that hidden state as the
        # input for the next minibatch. In this way, we artificially
        # preserve context between batches.
        loss, h0 = rnn.step(h0, x_oh, y_oh)

        average_loss += loss

        if batch % sample_every == 0: 
            # run sampling (2.2)
            fmtstr = 'epoch={}/{}, batch={}/{}, batch loss={:.3f}, cumulative loss={:.3f}'
            #print("Epoch = ",current_epoch,"/",max_epochs,", Batch ",batch,"/",dataset.num_batches,", Loss = ",average_loss)
            print(fmtstr.format(current_epoch, max_epochs, batch, dataset.num_batches, loss, average_loss/batch))
            #sample = rnn.sample(dataset.encode("HAN:\nIs that good or bad?\n\n"), 200)
            #print(''.join(dataset.decode(sample)))
            
        batch += 1


In [ ]:

if __name__ == '__main__':
    np.random.seed(10)
    dataset = Dataset(minibatch_size=5, sequence_length=30)
    dataset.preprocess('data/selected_conversations.txt')
    dataset.create_minibatches()
    run_language_model(dataset, 1000, sequence_length=dataset.sequence_length)
